<a href="https://colab.research.google.com/github/KettoMisaell/UFC_logistic_prediction/blob/main/UFC_bouts_regresion_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Predicciones de los ganadores de combates de UFC.***

> Bloc con sangría



Se utilizará la regresión logística para decidir el ganador de cada combate de UFC registrado en el Data Set de Kaggle https://www.kaggle.com/datasets/mdabbert/ultimate-ufc-dataset, con 119 características diferentes por cada combate, estadísticas, de antes y después del combate, por supuesto para predecir el ganador solamente se ocuparán características de antes del combate.

También se hará uso de un pipeline para ejemplificar una buena práctica al momento de crear este tipo de modelos.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocesado y modelado
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
ufc = pd.read_csv("/content/ufc-master.csv")

In [ ]:
ufc

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,Thiago Santos,Johnny Walker,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,...,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0
1,Alex Oliveira,Niko Price,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0
2,Misha Cirkunov,Krzysztof Jotko,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0
3,Alexander Hernandez,Mike Breeden,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,...,Punch,1.0,1:20,80.0,175.0,900.0,500.0,3500.0,110.0,1100.0
4,Joe Solecki,Jared Gordon,-135.0,115,74.074074,115.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,165.0,200.0,400.0,1200.0,900.0,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,Duane Ludwig,Darren Elkins,-155.0,135,64.516129,135.000000,3/21/2010,"Broomfield, Colorado, USA",USA,Blue,...,NaN,1.0,0:44,44.0,NaN,NaN,NaN,NaN,NaN,NaN
4892,John Howard,Daniel Roberts,-210.0,175,47.619048,175.000000,3/21/2010,"Broomfield, Colorado, USA",USA,Red,...,Punch,1.0,2:01,121.0,NaN,NaN,NaN,NaN,NaN,NaN
4893,Brendan Schaub,Chase Gormley,-260.0,220,38.461538,220.000000,3/21/2010,"Broomfield, Colorado, USA",USA,Red,...,Punches,1.0,0:47,47.0,NaN,NaN,NaN,NaN,NaN,NaN
4894,Mike Pierce,Julio Paulino,-420.0,335,23.809524,335.000000,3/21/2010,"Broomfield, Colorado, USA",USA,Red,...,NaN,3.0,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN


Son 4896 filas, o sea 4896 combates distintos.

La columna en dónde se indica el ganador de dicho combate contiene dos posibles valores, 'Red' o 'Blue', dependiendo en la esquina en la que estaba peleando el ganador, para poder usarlo cómo target, se convertirá en 1 si el ganador fue la esquina Red o 0 si fue la esquina Blue.

In [ ]:
ufc['Winner'] = np.where(ufc['Winner'] == 'Red', 1, 0)

print("Número de observaciones por clase")
print(ufc['Winner'].value_counts())
print("")

print("Porcentaje de observaciones por clase")
print(100 * ufc['Winner'].value_counts(normalize=True))

Número de observaciones por clase
1    2859
0    2037
Name: Winner, dtype: int64

Porcentaje de observaciones por clase
1    58.394608
0    41.605392
Name: Winner, dtype: float64


El 58 por ciento de los combates lo ganó la esquina roja, mientras que el 42 fue para la azul, un modelo de clasificación que sea útil debe de ser capaz de predecir correctamente un porcentaje de observaciones por encima del porcentaje de la clase mayoritaria.

Cómo primera aproximación se utilizarán 6 características, las apuestas previo al combate, y el record de cada peleador, es decir sus peleas ganadas y peleas perdidas.

In [ ]:
X = ufc[['R_odds', 'B_odds', 'R_wins', 'B_wins', 'R_losses', 'B_losses']]
y = ufc['Winner']

In [ ]:
X.isnull().sum()

R_odds      1
B_odds      0
R_wins      0
B_wins      0
R_losses    0
B_losses    0
dtype: int64

In [ ]:
X.fillna(0, inplace = True)

<ipython-input-15-1272d6d86a40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace = True)


In [ ]:
X.isnull().sum()

R_odds      0
B_odds      0
R_wins      0
B_wins      0
R_losses    0
B_losses    0
dtype: int64

Se utilizará StandardScaler de la libreria sklearn para normalizar los datos y que las diferentes escalas no afecten al modelo.

In [ ]:
from sklearn.preprocessing import StandardScaler


# Crea el objeto de la clase StandardScaler
scaler = StandardScaler()

# Normaliza cada columna del dataframe por separado
for column in X.columns:
    X[column] = scaler.fit_transform(X[column].values.reshape(-1, 1))

<ipython-input-18-41d4c68e5991>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = scaler.fit_transform(X[column].values.reshape(-1, 1))
<ipython-input-18-41d4c68e5991>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = scaler.fit_transform(X[column].values.reshape(-1, 1))
<ipython-input-18-41d4c68e5991>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

Se separa la data de entrenamiento y prueba.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

Para calcular el término de intercepto (constante) en el modelo, se agregará en la primera linea una columna constante de unos al conjunto de entrenamiento.

In [ ]:
X_train = sm.add_constant(X_train, prepend=True) #Se agrega la columna constante
modelo = sm.Logit(endog=y_train, exog=X_train,) #Creación del modelo
modelo = modelo.fit() #Ajuste
print(modelo.summary())

Optimization terminated successfully.
         Current function value: 0.617858
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 3916
Model:                          Logit   Df Residuals:                     3909
Method:                           MLE   Df Model:                            6
Date:                Fri, 21 Jul 2023   Pseudo R-squ.:                 0.09000
Time:                        19:18:53   Log-Likelihood:                -2419.5
converged:                       True   LL-Null:                       -2658.8
Covariance Type:            nonrobust   LLR p-value:                3.462e-100
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3837      0.035     10.988      0.000       0.315       0.452
R_odds        -0.5055      0.

Se hacen las predicciones del modelo con la Data de entrenamiento.

In [ ]:
predicciones = modelo.predict(exog = X_train)
#Se indica que si la probabilidad es mayor a 0.5 se clasifique en la categoría con mayor probabilidad
clasificacion = np.where(predicciones<0.5, 0, 1)
clasificacion

array([1, 1, 0, ..., 1, 0, 1])

Se testea que tan preciso es el modelo preciendo y comparando con la Data que se separó especificamente para hacer la prueba de que tan preciso es.

In [ ]:
X_test = sm.add_constant(X_test, prepend=True)
predicciones = modelo.predict(exog = X_test)
clasificacion = np.where(predicciones<0.5, 0, 1)
accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = clasificacion,
            normalize = True
           )
print("")
print(f"El modelo tiene un porcentaje de precision de: {100*accuracy}%")


El accuracy de test es: 65.20408163265307%


Parece que al menos tiene un porcentaje de precision mayor al de porcentaje de la clase mayoritaria, aún así, no parece ser muy preciso.

Se agregaran 4 nuevas caracteristicas al modelo, las edades de los peleadores, y sus rachas.

In [ ]:
X = ufc[['R_odds', 'B_odds', 'R_wins', 'B_wins', 'R_losses', 'B_losses', 'B_current_win_streak', 'R_current_win_streak', 'B_age', 'R_age' ]]
y = ufc['Winner']

In [ ]:
X.isnull().sum()

R_odds                  1
B_odds                  0
R_wins                  0
B_wins                  0
R_losses                0
B_losses                0
B_current_win_streak    0
R_current_win_streak    0
B_age                   0
R_age                   0
dtype: int64

In [ ]:
X.fillna(0, inplace = True)

<ipython-input-9-1272d6d86a40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace = True)


Se repiten los procesos anteriores.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [ ]:
X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.Logit(endog=y_train, exog=X_train,)
modelo = modelo.fit()
print(modelo.summary())

Optimization terminated successfully.
         Current function value: 0.615333
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 3916
Model:                          Logit   Df Residuals:                     3905
Method:                           MLE   Df Model:                           10
Date:                Sun, 23 Jul 2023   Pseudo R-squ.:                 0.09372
Time:                        02:03:24   Log-Likelihood:                -2409.6
converged:                       True   LL-Null:                       -2658.8
Covariance Type:            nonrobust   LLR p-value:                9.927e-101
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.1184      0.375      0.316      0.752      -0.617       0.854

In [ ]:
predicciones = modelo.predict(exog = X_train)

clasificacion = np.where(predicciones<0.5, 0, 1)
clasificacion

array([1, 1, 0, ..., 1, 1, 1])

In [ ]:
X_test = sm.add_constant(X_test, prepend=True)
predicciones = modelo.predict(exog = X_test)
clasificacion = np.where(predicciones<0.5, 0, 1)
accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = clasificacion,
            normalize = True
           )
print("")
print(f"El modelo tuvo un porcentaje de precision de : {100*accuracy}%")


El modelo tuvo un porcentaje de precision de : 65.10204081632654%


Al parecer la precisión del modelo no aumento aunque se hayan agregado estas características que tenían los peleadores previo a la pelea, lo que se podría interpretar cómo que una racha ganadora larga y un buen record previo a una pelea no son parámetros muy significativos para tratar de predecir si ese peleador ganará el combate.

Por último, llegados a este punto podrán haber notado que estoy cometiendo una mala práctica al usar código repetitivo, y no reutilizarlo, por eso, los pasos anteriores los resumí en un pipeline para generar el modelo, y así cada vez que quiero cambiar las variables independientes para hacer experimentos con la predicción no tenga que volver a escribir todo el código, y de igual forma facilitando su lectura.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#Variables y target
ufc.B_avg_TD_landed
X = ufc[['R_ev', 'B_ev','R_total_title_bouts', 'B_total_title_bouts']]
X.fillna(0, inplace = True)
y = ufc['Winner']

# Se divide la Data en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Se crea el modelo y se aplica la normalización usando un pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalización
    ('model', LogisticRegression())  # Se indica que es un modelo de regresión logística
])

# Se ajusta el modelo
pipeline.fit(X_train, y_train)

# Predicciones
y_pred = pipeline.predict(X_test)

# Se evalua la precisión del modelo.
accuracy = accuracy_score(y_test, y_pred)
print("")
print(f"El modelo tuvo un porcentaje de precision de : {100*accuracy}%")



El modelo tuvo un porcentaje de precision de : 65.0%


<ipython-input-17-09af0a4cd788>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace = True)


En este último modelo sustituí las variables independientes anteriores dejando las apuestas previo a la pelea y poniendo cómo nuevas variables el número de peleas por el titulo que habían tenido estos peleadores, sin embargo la respuesta fue muy similar a las anteriores por lo que puedo concluir que las variables más significativas a tomar en cuenta para predecir el resultado de un combate (al menos con la información disponible en este Dataset) son las apuestas previo al combate, suena algo logico porque generalmente los apostadores por lo general toman en cuenta estas y más variables antes de arriesgar su dinero.